In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"  # specify which GPU(s) to be used

import warnings
warnings.filterwarnings('ignore')

import mlflow

EXPERIMENT_NAME = 'EAT Prediction'
mlflow.set_tracking_uri("http://0.0.0.0:41250")
mlflow.set_experiment(EXPERIMENT_NAME)

In [2]:
import ipywidgets as widgets
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
import time
import gc
import json
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import Callback 
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors

tf.get_logger().setLevel('ERROR')

# Basic functions

In [3]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

In [4]:
def build_model(transformer, max_len=512, hidden_dim=32, n_classes=3):
    assert n_classes > 2 # only multiclass
    
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    
    out = Dense(n_classes, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
def train_model(n_steps, train_dataset, model):
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='loss', 
        verbose=1,
        patience=1,
        mode='min',
        restore_best_weights=True
    )
    
    train_history = model.fit(
        train_dataset,
        steps_per_epoch=n_steps,
        epochs=EPOCHS,
        callbacks=[early_stopping]
    )
    return model, train_history

In [6]:
def evaluate_model(ids, test_dataset, y_true_encoded, model):
    y_pred_encoded = np.argmax(
        model.predict(test_dataset, verbose=1),
        axis=1)
    
    assert len(y_true_encoded) == len(y_pred_encoded)
    
    f1 = f1_score(y_true_encoded, y_pred_encoded, average='weighted')
    precision = precision_score(y_true_encoded, y_pred_encoded, average='weighted')
    recall = recall_score(y_true_encoded, y_pred_encoded, average='weighted')
    accuracy = accuracy_score(y_true_encoded, y_pred_encoded)
    
    # getting correctly and incorrectly classified questions
    results = list()
    for i in range(len(y_true_encoded)):
        if y_pred_encoded[i] != y_true_encoded[i]:
            results.append('+')
        else:
            results.append('-')
            
    df = pd.DataFrame.from_dict({'id': ids, 'result': results, 'true': y_true_encoded, 'pred': y_pred_encoded})
    
    return f1, precision, recall, accuracy, df

# GPU Check

In [7]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print("---------- Available GPUs:", get_available_gpus(), "--------------")

---------- Available GPUs: ['/device:GPU:0', '/device:GPU:1'] --------------


# Hyperparameter setup

In [8]:
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
EPOCHS = 8
BATCH_SIZE = 8 * len(get_available_gpus())
MAX_LEN = 128
MODEL = 'bert-base-cased'

TRAIN_LIST = ['LC-QuAD',
                'QALD',
                'CogComp',
                'WebQuestions',
                'SimpleQuestions',
                'LC-QuAD+QALD',
                'LC-QuAD+CogComp',
                'LC-QuAD+WebQuestions',
                'LC-QuAD+SimpleQuestions',
                'QALD+CogComp',
                'QALD+WebQuestions',
                'QALD+SimpleQuestions',
                'CogComp+WebQuestions',
                'CogComp+SimpleQuestions',
                'WebQuestions+SimpleQuestions',
                'LC-QuAD+QALD+CogComp',
                'LC-QuAD+QALD+WebQuestions',
                'LC-QuAD+QALD+SimpleQuestions',
                'LC-QuAD+CogComp+WebQuestions',
                'LC-QuAD+CogComp+SimpleQuestions',
                'LC-QuAD+WebQuestions+SimpleQuestions',
                'QALD+CogComp+WebQuestions',
                'QALD+CogComp+SimpleQuestions',
                'QALD+WebQuestions+SimpleQuestions',
                'CogComp+WebQuestions+SimpleQuestions',
                'LC-QuAD+QALD+CogComp+WebQuestions',
                'LC-QuAD+QALD+CogComp+SimpleQuestions',
                'LC-QuAD+QALD+WebQuestions+SimpleQuestions',
                'LC-QuAD+CogComp+WebQuestions+SimpleQuestions',
                'QALD+CogComp+WebQuestions+SimpleQuestions',
                'LC-QuAD+QALD+CogComp+WebQuestions+SimpleQuestions']

TEST_LIST = ['LC-QuAD', 'QALD', 'CogComp', 'WebQuestions', 'SimpleQuestions'] 

# classes
types = ['Event', 'Place', 'Colour', 'SportsSeason', 'Name', 'DateTime', 'Device', 'Activity', 'Number', 'Biomolecule', 'Disease', 'Food', 'Work', 'AnatomicalStructure', 'Currency', 'TopicalConcept', 'Species', 'Boolean', 'Award', 'TimePeriod', 'Altitude', 'Agent', 'Language', 'Flag', 'Holiday', 'ChemicalSubstance', 'MeanOfTransportation', 'Medicine', 'EthnicGroup', 'PersonFunction', 'String', 'List']

# Create Tokenizer

In [9]:
# First load the real tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# The Pipeline

In [10]:
def pipeline():
    encoder = LabelEncoder()
    encoder.fit(types)
        
    for train_name in TRAIN_LIST:
        train_df = pd.read_csv(f"../../data/UnifiedSubclassDBpedia/{train_name}-train.csv", sep=';')
        
        encoded_y_train = encoder.transform(train_df.type)
        dummy_y_train = np_utils.to_categorical(encoded_y_train)
        
        x_train = regular_encode(train_df.questionText.astype(str), tokenizer, maxlen=MAX_LEN)
        
        train_dataset = (
            tf.data.Dataset
            .from_tensor_slices((x_train, dummy_y_train))
            .repeat()
            .shuffle(2048)
            .batch(BATCH_SIZE)
            .prefetch(AUTO)
        )
        
        transformer_layer = TFAutoModel.from_pretrained(MODEL)
        model = build_model(transformer_layer, max_len=MAX_LEN, n_classes=len(types))
        n_steps = x_train.shape[0] // BATCH_SIZE
        
        print(f"--------- START TRAINING {train_name} ------------")
        start = time.time()
        ########
        model, train_history = train_model(n_steps, train_dataset, model)
        ########
        train_time = time.time() - start
        print(f"--------- END TRAINING {train_name}------------")
        
        with open(f"../../data/experimental_metadata/{train_name}.hist", 'w') as file:
            json.dump(train_history.history, file, indent=4)
        
        for test_name in TEST_LIST:
            test_df = pd.read_csv(f"../../data/UnifiedSubclassDBpedia/{test_name}-test.csv", sep=';')
        
            x_test = regular_encode(test_df.questionText.astype(str), tokenizer, maxlen=MAX_LEN)
            y_true_encoded = encoder.transform(test_df.type)
            
            test_dataset = (
                tf.data.Dataset
                .from_tensor_slices(x_test)
                .batch(BATCH_SIZE)
            )
            
            print(f"--------- START EVALUATION {test_name} ------------")
            start = time.time()
            #########
            f1, precision, recall, accuracy, df_meta = evaluate_model(
                test_df.question,
                test_dataset,
                y_true_encoded,
                model
            )
            #########
            inference_time = time.time() - start
            print(f"--------- END EVALUATION {test_name} ------------")
            
            df_meta.to_csv(
                f"../../data/experimental_metadata/TRAIN:{train_name} || TEST:{test_name}.csv",
                sep=';',
                index=False
            )
            
            with mlflow.start_run():
                mlflow.log_param("Train Data", train_name)
                mlflow.log_param("Test Data", test_name)
                mlflow.log_param("EPOCHS", EPOCHS)
                mlflow.log_param("BATCH_SIZE", BATCH_SIZE)
                mlflow.log_param("MAX_LEN", MAX_LEN)
                mlflow.log_param("MODEL", MODEL)
                mlflow.log_param("Metadata Path", f"/home/aperevalov/hs-anhalt-master-thesis/data/experimental_metadata/TRAIN:{train_name} || TEST:{test_name}.csv")
                mlflow.log_param("History Path", f"/home/aperevalov/hs-anhalt-master-thesis/data/experimental_metadata/{train_name}.hist")
                mlflow.log_metric("Accuracy", accuracy)
                mlflow.log_metric("Precision", precision)
                mlflow.log_metric("Recall", recall)
                mlflow.log_metric("F1 Score", f1)
                mlflow.log_metric("Training Time", train_time)
                mlflow.log_metric("Inference Time", inference_time)
                
                print("F1", f1)

In [11]:
pipeline()

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Some weights of the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were init

--------- START TRAINING LC-QuAD ------------
Epoch 1/8
116/116 [==============================] - 19s 161ms/step - loss: 2.1942 - accuracy: 0.2662
Epoch 2/8
116/116 [==============================] - 19s 161ms/step - loss: 1.8247 - accuracy: 0.3739
Epoch 3/8
116/116 [==============================] - 19s 160ms/step - loss: 1.2745 - accuracy: 0.5873
Epoch 4/8
116/116 [==============================] - 19s 160ms/step - loss: 0.6905 - accuracy: 0.8766
Epoch 5/8
116/116 [==============================] - 19s 160ms/step - loss: 0.5149 - accuracy: 0.9100
Epoch 6/8
116/116 [==============================] - 18s 159ms/step - loss: 0.3512 - accuracy: 0.9461
Epoch 7/8
116/116 [==============================] - 18s 159ms/step - loss: 0.2894 - accuracy: 0.9542
Epoch 8/8
116/116 [==============================] - 18s 159ms/step - loss: 0.2256 - accuracy: 0.9628


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


--------- END TRAINING LC-QuAD------------
--------- START EVALUATION LC-QuAD ------------
29/29 [==============================] - 2s 59ms/step
--------- END EVALUATION LC-QuAD ------------


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


F1 0.4140262284160063
--------- START EVALUATION WebQuestions ------------
 55/172 [========>.....................] - ETA: 7s

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/aperevalov/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-29bc503c7850>", line 1, in <module>
    pipeline()
  File "<ipython-input-10-e0dbe04b49fc>", line 56, in pipeline
    model
  File "<ipython-input-6-d92cf7f350e3>", line 3, in evaluate_model
    model.predict(test_dataset, verbose=1),
  File "/home/aperevalov/miniconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 130, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/home/aperevalov/miniconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1599, in predict
    tmp_batch_outputs = predict_function(iterator)
  File "/home/aperevalov/miniconda3/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py", line 780, in __call__
    result = self._call(*args, **kwds)


KeyboardInterrupt: 